In [1]:
import azureml.core
from azureml.core import Workspace, Datastore
from azureml.data import OutputFileDatasetConfig


ws = Workspace(subscription_id = '6ed9d167-b2e6-41b8-9500-35e6df64d9dc',
                resource_group = 'MLRG',
                workspace_name = 'erbbimlws'
              )

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

In [3]:
from azureml.pipeline.core.graph import PipelineParameter

pipeline_param = PipelineParameter(
  name="pipeline_arg",
  default_value=10)

In [4]:
compute_name = "vkontogCompute"

In [5]:
# Default datastore 
datastore= Datastore(ws, "workspaceblobstore")
#datastore= Datastore(ws, "workspacefilestore")

In [6]:
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    print('Please set up a proper compute')


In [7]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 


aml_run_config = RunConfiguration()
# Use just-specified compute target ("cpu-cluster")
aml_run_config.target = compute_target

In [8]:
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['scikit-learn','numpy',' xgboost'], 
    pip_packages=['azureml-sdk', 'azureml-dataset-runtime[fuse,pandas]'], 
    pin_sdk_version=False)

In [9]:
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData



In [48]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

#dataprep_step0 = PythonScriptStep(
#    name="Spacy_NLP_v0", 
#    script_name="./Text_Mining_withSpacy/pipTest.py", 
#    compute_target=compute_target, 
#    runconfig=aml_run_config,
#    allow_reuse=True
#)

dataprep_step1 = PythonScriptStep(
    name="vBankingPB_XGBoost_s1", 
    script_name="./vBankingPB_XGBoost_s1.py", 
    compute_target=compute_target, 
    runconfig=aml_run_config,
    allow_reuse=False
    
    )

In [49]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment

pipeline = Pipeline(ws, [dataprep_step1])

experiment = Experiment(workspace=ws, name='vBanking_PB_Pipeline')

azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 1000000000

run = experiment.submit(pipeline, show_output=True)
run.wait_for_completion()

Created step vBankingPB_XGBoost_s1 [0de7074b][b2caea92-8c67-4c1c-adcd-20514d88b390], (This step will run and generate new outputs)
Submitted PipelineRun 2e22a806-fa5e-4e3d-92f6-4ef863594140
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2e22a806-fa5e-4e3d-92f6-4ef863594140?wsid=/subscriptions/6ed9d167-b2e6-41b8-9500-35e6df64d9dc/resourcegroups/MLRG/workspaces/erbbimlws&tid=c1635f99-0e6e-4dac-89a9-1b9c473202f4
PipelineRunId: 2e22a806-fa5e-4e3d-92f6-4ef863594140
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2e22a806-fa5e-4e3d-92f6-4ef863594140?wsid=/subscriptions/6ed9d167-b2e6-41b8-9500-35e6df64d9dc/resourcegroups/MLRG/workspaces/erbbimlws&tid=c1635f99-0e6e-4dac-89a9-1b9c473202f4
PipelineRun Status: NotStarted
PipelineRun Status: Running




PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': '2e22a806-fa5e-4e3d-92f6-4ef863594140', 'status': 'Completed', 'startTimeUtc': '2022-03-31T13:47:54.802203Z', 'endTimeUtc': '2022-03-31T13:49:38

Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


'Finished'